In [1]:
from os import listdir
import numpy as np
import pandas as pd

In [2]:
my_path = 'Dataset/Classification-Train And Test/'
train_path = my_path + 'Train/'
test_path  = my_path + 'Test/'

In [3]:
def get_files_list(directory_path):
    rows = pd.DataFrame(columns=['path', 'class'])
    for class_path_name in listdir(directory_path):
        for data_file in listdir(directory_path + class_path_name):
            single_row = pd.DataFrame ({
                'path' : [directory_path + class_path_name + '/' + data_file],
                'class' : [class_path_name]
            })
            rows = pd.concat([rows, single_row], ignore_index=True)
    return rows

In [4]:
doc_train = get_files_list(train_path)
doc_test  = get_files_list(test_path)

In [61]:
doc_test

,path,class
0,Dataset/Classification-Train And Test/Test/Com...,Comp.graphics
1,Dataset/Classification-Train And Test/Test/Com...,Comp.graphics
2,Dataset/Classification-Train And Test/Test/rec...,rec.autos
3,Dataset/Classification-Train And Test/Test/rec...,rec.autos
4,Dataset/Classification-Train And Test/Test/rec...,rec.autos
5,Dataset/Classification-Train And Test/Test/sci...,sci.electronics
6,Dataset/Classification-Train And Test/Test/sci...,sci.electronics
7,Dataset/Classification-Train And Test/Test/sci...,sci.electronics
8,Dataset/Classification-Train And Test/Test/soc...,soc.religion.christian
9,Dataset/Classification-Train And Test/Test/soc...,soc.religion.christian


In [22]:
doc_test

,path,class
0,Dataset/Classification-Train And Test/Test/Com...,Comp.graphics
1,Dataset/Classification-Train And Test/Test/Com...,Comp.graphics
2,Dataset/Classification-Train And Test/Test/rec...,rec.autos
3,Dataset/Classification-Train And Test/Test/rec...,rec.autos
4,Dataset/Classification-Train And Test/Test/rec...,rec.autos
5,Dataset/Classification-Train And Test/Test/sci...,sci.electronics
6,Dataset/Classification-Train And Test/Test/sci...,sci.electronics
7,Dataset/Classification-Train And Test/Test/sci...,sci.electronics
8,Dataset/Classification-Train And Test/Test/soc...,soc.religion.christian
9,Dataset/Classification-Train And Test/Test/soc...,soc.religion.christian


In [7]:
def tokenize_sentence(line):
    return line.strip().split(" ")

In [8]:
def tokenize(document_path):
    f = open(document_path, 'r')
    text_lines_in_given_document = f.readlines()

    words_in_given_document = []

    #traverse over all the lines and tokenize each one with the help of helper function: tokenize_sentence
    for line in text_lines_in_given_document:
        words_in_given_document.append(tokenize_sentence(line))

    return words_in_given_document

In [9]:
#a simple helper function to convert a 2D array to 1D, without using numpy
def flatten(list):
    new_list = []
    for i in list:
        for j in i:
            new_list.append(j)
    return new_list

In [10]:
list_of_words_in_all_project_train_data_in_a_nested_structure = []
list_of_words_in_all_project_test_data_in_a_nested_structure = []

for document, row in doc_train.iterrows():
    list_of_words_in_all_project_train_data_in_a_nested_structure.append(
        flatten(
            tokenize(row['path'])
        )
    )


for document, row in doc_test.iterrows():
    list_of_words_in_all_project_test_data_in_a_nested_structure.append(
        flatten(
            tokenize(row['path'])
        )
    )

In [53]:
list_of_words_in_all_project_train_data_in_1D_list = flatten(
    list_of_words_in_all_project_train_data_in_a_nested_structure
)

np_list_of_words = np.asarray(
    list_of_words_in_all_project_train_data_in_1D_list
)

df = pd.DataFrame({"Word": np_list_of_words})
train_bag_of_words = df['Word'].value_counts().reset_index()

# Rename columns for clarity
train_bag_of_words.columns = ['Word', 'Count']

train_dictionary = {}
document_number = 1
for words_in_a_document in list_of_words_in_all_project_train_data_in_a_nested_structure:
    #print(doc_words)
    np_doc_words = np.asarray(words_in_a_document) #words in one document in a ndarray
    unique_words_in_one_document, counts_of_unique_words_in_one_document = np.unique(np_doc_words, return_counts=True)
    train_dictionary[document_number] = {}
    for i in range(len(unique_words_in_one_document)):
        train_dictionary[document_number][unique_words_in_one_document[i]] = counts_of_unique_words_in_one_document[i]
    document_number = document_number + 1

train_features = train_bag_of_words


#now we make a 2D array having the frequency of each word of our feature set in each individual documents

X_train = []

for document_num in train_dictionary.keys():
    row = []
    for index, feature_word in train_features.iterrows():
        if feature_word['Word'] in train_dictionary[document_num].keys():
            #if word is present in the dictionary of the document as a key, its value is copied
            #this gives us no. of occurences
            row.append(train_dictionary[document_num][feature_word['Word']])
        else:
            #if not present, the no. of occurences is zero
            row.append(0)
    X_train.append(row)



X_train = np.asarray(X_train)
Y_train = np.asarray(doc_train['class'])

In [60]:
X_train

array([[ 27,  14,   5, ...,   0,   0,   0],
       [ 45,  21,  18, ...,   0,   0,   0],
       [ 40,  14,  13, ...,   0,   0,   0],
       ...,
       [ 64,  31,  25, ...,   0,   0,   0],
       [272, 146,  71, ...,   0,   0,   0],
       [135,  71,  49, ...,   0,   0,   1]], dtype=int64)

In [58]:
test_dictionary = {}
document_number = 1
for words_in_a_document in list_of_words_in_all_project_test_data_in_a_nested_structure:
    #print(doc_words)
    np_doc_words = np.asarray(words_in_a_document) #words in one document in a ndarray
    unique_words_in_one_document, counts_of_unique_words_in_one_document = np.unique(np_doc_words, return_counts=True)
    test_dictionary[document_number] = {}
    for i in range(len(unique_words_in_one_document)):
        test_dictionary[document_number][unique_words_in_one_document[i]] = counts_of_unique_words_in_one_document[i]
    document_number = document_number + 1

# train_features = train_bag_of_words


#now we make a 2D array having the frequency of each word of our feature set in each individual documents

X_test = []

for document_num in test_dictionary.keys():
    row = []
    for index, feature_word in train_features.iterrows():
        if feature_word['Word'] in test_dictionary[document_num].keys():
            #if word is present in the dictionary of the document as a key, its value is copied
            #this gives us no. of occurences
            row.append(test_dictionary[document_num][feature_word['Word']])
        else:
            #if not present, the no. of occurences is zero
            row.append(0)
    X_test.append(row)



X_test = np.asarray(X_test)
Y_test = np.asarray(doc_test['class'])

In [59]:
X_test

array([[  8,   6,  12, ...,   0,   0,   0],
       [ 22,   7,   7, ...,   0,   0,   0],
       [ 24,   7,   2, ...,   0,   0,   0],
       ...,
       [119,  71,  43, ...,   0,   0,   0],
       [ 50,  35,  20, ...,   0,   0,   1],
       [ 69,  31,  15, ...,   0,   0,   0]], dtype=int64)

In [62]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, Y_train)
display(X_train)

array([[ 27,  14,   5, ...,   0,   0,   0],
       [ 45,  21,  18, ...,   0,   0,   0],
       [ 40,  14,  13, ...,   0,   0,   0],
       ...,
       [ 64,  31,  25, ...,   0,   0,   0],
       [272, 146,  71, ...,   0,   0,   0],
       [135,  71,  49, ...,   0,   0,   1]], dtype=int64)

In [63]:
Y_predict = clf.predict(X_test)


In [64]:
clf.score(X_test, Y_test)


0.9411764705882353

In [87]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(Y_test, Y_predict))

                        precision    recall  f1-score   support

         Comp.graphics       1.00      0.50      0.67         2
             rec.autos       1.00      1.00      1.00         3
       sci.electronics       1.00      1.00      1.00         3
soc.religion.christian       1.00      1.00      1.00         4
 talk.politics.mideast       0.83      1.00      0.91         5

              accuracy                           0.94        17
             macro avg       0.97      0.90      0.92        17
          weighted avg       0.95      0.94      0.93        17

